In [1]:
import pandas as pd
import plotly         
import plotly.express as px
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc  
import plotly.io as pio
import statsmodels
from dash.dependencies import Input,Output,State

In [2]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [3]:
df = pd.read_excel("Salary and Months.xlsx")
df.head()

,Position,Month 1,Month 6,Month 12,Month 18,Month 24,Total Expenditure in two years
0,Manager,27917,111667,195417,251250,279167,865418
1,Video editor,20000,80000,120000,160000,180000,560000
2,Video editor(Interns),5000,20000,30000,40000,45000,140000
3,Contingency team(Video Editing),3000,12000,18000,24000,27000,84000
4,Content designer,20833,104167,145833,187500,229167,687500


In [4]:
df1 = pd.read_excel("CombinedData.xlsx")
df1.head()

,Position,Number of employees,Salary/month/employee,Salary/month,Salary/year/employee,Salary/year,Month 1,Month 2,Month 3,Month 4,...,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24
0,Manager,1,27916.67,27916.67,335000,335000,27917,27917,83750,83750,...,223333,223333,223333,251250,251250,251250,279167,279167,279167,279167
1,Video editor,1,20000.00,20000.00,240000,240000,20000,20000,60000,60000,...,140000,140000,160000,160000,160000,160000,180000,180000,180000,180000
2,Video editor(Interns),5,1000.00,5000.00,12000,60000,5000,5000,15000,15000,...,35000,35000,40000,40000,40000,40000,45000,45000,45000,45000
3,Contingency team(Video Editing),3,1000.00,3000.00,12000,36000,3000,3000,9000,9000,...,21000,21000,24000,24000,24000,24000,27000,27000,27000,27000
4,Content designer,1,20833.33,20833.33,250000,250000,20833,20833,62500,62500,...,187500,187500,187500,187500,208333,208333,208333,229167,229167,229167


In [5]:
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Hr(style={'height': '10px', 'color': 'blue'})
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.H1("Team - C (Salary Sheet - Creatives & HR)", 
                    style={'textAlign': 'center'})
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Hr(style={'height': '10px', 'color': 'blue'})
        ])
    ]),
#  ----------------------------------------------------------------------------------------------------------    
    
    dbc.Row([
        dbc.Col([
            html.H1("Calculator")
        ], width=6)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Label(['Select Positions'], 
                       style={'font-weight': 'bold', "text-align": "center"}),
            dcc.Dropdown(id='Positions',
                    options=[
                            {'label': 'Manager', 'value': 'Manager'},
                            {'label': 'Video editor', 'value': 'Video editor'},
                            {'label': 'Video editor(Interns)', 'value': 'Video editor(Interns)'},
                            {'label': 'Contingency team(Video Editing)	', 'value': 'Contingency team(Video Editing)	'},
                            {'label': 'Content designer', 'value': 'Content designer'},
                            {'label': 'Content designer(Intern)', 'value': 'Content designer(Intern)'},
                            {'label': 'Contingency team(Content Designing)', 'value': 'Contingency team(Content Designing)'},
                            {'label': 'Content writer', 'value': 'Content writer'},
                            {'label': 'Content writer(Intern)', 'value': 'Content writer(Intern)'},
                            {'label': 'Contingency team(Content writing)', 'value': 'Contingency team(Content writing)'},
                            {'label': 'HR manager', 'value': 'HR manager'},
                            {'label': 'Recruiter', 'value': 'Recruiter'}
                    ],
                    optionHeight=35,                    
                    value='Manager',                  
                    disabled=False,                    
                    multi=False,                       
                    searchable=True,                    
                    search_value='',                    
                    placeholder='Please select...',     
                    clearable=True,                     
                    style={'width':"100%"}),
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Br(),
            html.Label(['Enter the number of Employees'],
                       style={'font-weight': 'bold', "text-align": "left"}), 
            dcc.Input(id='no_of_emp',type='number',placeholder='',
                          style={'display':'inline-block'}),
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Label(['Calculator :'],
                           style={'font-weight': 'bold', "text-align": "center"}),
            html.Br(),

            html.Label(['The Salary Per Month Per Employee is: ']),
            html.Div(id='output_container1', children=[]),
            
            html.Label(['The Salary Per Month is: ']),
            html.Div(id='output_container2', children=[]),
            
            html.Label(['The Salary Per Year Per Employee is: ']),
            html.Div(id='output_container3', children=[]),
            
            html.Label(['The Salary Per Year is: ']),
            html.Div(id='output_container4', children=[]),
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Hr(style={'height': '10px', 'color': 'blue'})
        ])
    ]),
    
#  ----------------------------------------------------------------------------------------------------------
   
    
    dbc.Row([
        dbc.Col([
            html.H1("Expenditure of Ozibook Ltd")
        ], width=6)
    ]),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='Graph3')
        ], width=8),
        
        dbc.Col([
            html.Label(['Y-axis values to compare:'], 
                       style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='yaxis_raditem',
                options=[
                         {'label': 'Month 1', 'value': 'Month 1'},
                         {'label': 'Month 6', 'value': 'Month 6'},
                         {'label': 'Month 12', 'value': 'Month 12'},
                         {'label': 'Month 18', 'value': 'Month 18'},
                         {'label': 'Month 24', 'value': 'Month 24'},
                         {'label': 'Total Expenditure in Two years', 'value': 'Total Expenditure in two years'}
                ],
                value='Month 1'
            )
        ], width=4)
    ])
])


In [6]:
@app.callback(
    Output(component_id='output_container1', component_property='children'),
    Output(component_id='output_container2', component_property='children'),
    Output(component_id='output_container3', component_property='children'),
    Output(component_id='output_container4', component_property='children'),
    [Input('no_of_emp','value'),
    Input('Positions','value')]
)

def calculator_output(no_of_emp,Position):
    data = df1.copy()
    data = data[data['Position']==Position]
    x1=data['Salary/month/employee']
    x2=data['Salary/year/employee'] 
    s_per_emp = x1
    s_emp = x1*no_of_emp
    s_per_year = x2
    s_year = x2*no_of_emp
    
    return s_per_emp, s_emp, s_per_year, s_year

# ----------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id='Graph3', component_property='figure'),
    [Input(component_id='yaxis_raditem', component_property='value')]
)

def update_graph(y_axis):
    dff = df
    barchart=px.bar(
            data_frame=dff,
            x="Position",
            y=y_axis,
            title=y_axis+': By Position',
            color="Position",
            barmode="group",
            template='plotly_white',
            )
    barchart.update_layout(xaxis={'categoryorder':'total descending'},
                           title={'xanchor':'center', 'yanchor': 'top', 'y':0.9,'x':0.5,})
    
    
    return barchart
# ----------------------------------------------------------------------------------------------------------------

In [7]:
if __name__ == '__main__':
    app.run_server(port=8030)

Dash is running on http://127.0.0.1:8030/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8030
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1668414623.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET /_dash-component-suites/dash/deps/react@16.v2_7_0m1668414623.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_0m1668414623.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1668414623.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1659298715.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:30] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1668414623.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\dash\

127.0.0.1 - - [01/Dec/2022 20:31:31] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Dec/2022 20:31:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2022 20:31:36] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  File "C:\Users\SHREYASH SALUNKE\AppData\Roaming\Python\Python310\site-packages\dash\

127.0.0.1 - - [01/Dec/2022 20:31:36] "POST /_dash-update-component HTTP/1.1" 500 -
